In [1]:
import pandas as pd
from konlpy.tag import Okt
from gensim.models import Word2Vec
import numpy as np

In [2]:
df = pd.read_excel("Training.xlsx")

In [3]:
df.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)

In [4]:
df.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)


In [5]:
df_train = pd.concat([
    df[['Emotion', 'sentence']],
    df[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)



In [6]:
df_train = df_train.dropna()

In [7]:
okt = Okt()

temp = []
for i in df_train['sentence']:
    temp.append(okt.morphs(i))

df_train['token'] = temp

df_train.to_csv('training_data-token.csv', index=False)

In [9]:
model = Word2Vec(df_train['token'], vector_size=1000, window=5, min_count=1, workers=4, sg=1)

In [ ]:
model.wv.vectors.shape

In [ ]:
import joblib

joblib.dump(model, '../model/word_model.joblib')

In [ ]:
import pandas as pd

# 데이터를 불러옵니다.
data = pd.read_csv('training_data-token.csv')  # 데이터 파일 경로에 맞게 수정하세요.

# 감정 텍스트를 숫자로 변환하는 매핑
emotion_mapping = {
    '기쁨': 0,
    '당황': 1,
    '불안': 2,
    '슬픔': 3,
    '상처': 4,
    '분노': 5
}

# 'Emotion' 컬럼을 숫자로 변환
data['Emotion'] = data['Emotion'].map(emotion_mapping)

# 숫자로 변환된 감정 값에 맞게 샘플링을 설정합니다.
desired_ratio = {
    0: 16000,  # 기쁨 4.5
    3: 16000,  # 슬픔 2.0
    2: 16000,  # 불안 2.0
    4: 16000,  # 상처 1.5
    1: 16000,  # 당황 1.0
    5: 16000,  # 분노 1.0
}
# 선택한 데이터를 저장할 빈 데이터프레임을 생성합니다.
selected_data = pd.DataFrame(columns=data.columns)

# 각각의 감정에 대해 적절한 비율로 데이터를 선택합니다.
for emotion, ratio in desired_ratio.items():
    emotion_data = data[data['Emotion'] == emotion].sample(n=ratio, random_state=42)  # 적절한 비율로 샘플링
    selected_data = pd.concat([selected_data, emotion_data], ignore_index=True)  # 데이터프레임을 합칩니다.

# 선택된 데이터를 확인합니다.
print(selected_data)

# 선택된 데이터를 새로운 파일로 저장합니다.
selected_data.to_csv('selected_data.csv', index=False)  # 저장할 파일명에 맞게 수정하세요.


In [12]:
def get_sent_embedding(model, embedding_size, tokenized_words):
    feature_vec = np.zeros((embedding_size,), dtype="float32")
    n_words = 0
    for word in tokenized_words:
        if word in model.wv.key_to_index:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [14]:
df_train = pd.read_csv('selected_data.csv')

In [ ]:
from sklearn.svm import SVC
from joblib import load, dump

model = load("../model/word_model.joblib")

X = [get_sent_embedding(model, 1000, tokens) for tokens in df_train['token']]
y = df_train['Emotion']

# SVM 분류 모델 학습
svm_model = SVC(kernel='linear')
svm_model.fit(X, y)

# 모델 저장
dump(svm_model, '../model/svm_model.joblib')
